In [2]:
# ---------------------------------------------------------------
# 1 | Daten laden
# ---------------------------------------------------------------
import pandas as pd
from pathlib import Path

# Haupt-DF aus dem Clean-Clustering-Notebook
df = pd.read_pickle("de_final.pkl")          # enthält cluster_final etc.

# Ergebnisse aus dem Noise-Refinement
noise_refined = pd.read_pickle("noise_refined.pkl")  # enthält reassign_to

# ---------------------------------------------------------------
# 2 | Flag 'was_noise_reassigned' setzen
# ---------------------------------------------------------------
# Alle ID_gesamt, die aus Noise (-1) in ein echtes Cluster wandern
reassigned_ids = set(noise_refined.loc[
    noise_refined["reassign_to"].notna(), "ID_gesamt"
])

df["was_noise_reassigned"] = df["ID_gesamt"].isin(reassigned_ids)

# ---------------------------------------------------------------
# 3 | Export wie bisher – plus blaue Hervorhebung
# ---------------------------------------------------------------
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill

EXPORT_PATH = Path("cluster_ergebnis_plus_noise.xlsx")

# Farben
yellow_fill = PatternFill("solid", fgColor="FFFF99")   # wie gehabt
blue_fill   = PatternFill("solid", fgColor="ADD8E6")   # heller Blauton

export_cols = [
    "cluster_final",
    "ist_hauptfrage",
    "ähnlichkeit_zur_hauptfrage",
    "Frage_Text",
    "ID_gesamt",
]

# Gleiche Vorverarbeitung wie im Original-Export ...
df_export = df.copy().sort_values(
    ["cluster_final", "ist_hauptfrage", "ähnlichkeit_zur_hauptfrage"],
    ascending=[True, False, False]
)

# ---------------------------------------------------------------
wb = Workbook()
ws = wb.active
ws.title = "Smartvote Cluster"

bold = Font(bold=True)

# Überschriftenzeile
for col_idx, col in enumerate(export_cols, 1):
    ws.cell(1, col_idx, col).font = bold

row = 2
for _, r in df_export.iterrows():
    # Spalten schreiben
    for col_idx, col in enumerate(export_cols, 1):
        ws.cell(row, col_idx, r[col])

    # Gelb: normale Similarity-Hervorhebung
    if (
        r["ähnlichkeit_zur_hauptfrage"] < 1.0
        and not r["ist_hauptfrage"]
    ):
        for j in range(1, len(export_cols) + 1):
            ws.cell(row, j).fill = yellow_fill

    # Blau: neu zugeordnete Noise-Fragen
    if r["was_noise_reassigned"]:
        for j in range(1, len(export_cols) + 1):
            ws.cell(row, j).fill = blue_fill

    row += 1

wb.save(EXPORT_PATH)
print("✓ Excel exportiert:", EXPORT_PATH)


✓ Excel exportiert: cluster_ergebnis_plus_noise.xlsx
